In [1]:
from models.classifier import TransformerEncoder
import torch

/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hparams = {
    "hidden_size": 48,  # size of the hidden layers and embeddings
    "hidden_ff": 96,  # size of the position-wise feed-forward layer
    "n_encoders": 4,  # number of encoder blocks
    "n_heads": 2,  # number of attention heads in the multiheadattention module
    "n_local": 2,  # number of local attention heads
    "local_window_size": 4,  # size of the window for local attention
    'batch_size': 4,
    "max_length": 30,  # maximum length of the input sequence
    "vocab_size": 100,  # size of the vocabulary
    "learning_rate": 0.001,
    "num_epochs": 30,
    "attention_type": "performer",
    "norm_type": "rezero",
    "num_random_features": 32,  # number of random features for the Attention module (Performer uses this)
    "emb_dropout": 0.1,  # dropout for the embedding block
    "fw_dropout": 0.1,  # dropout for the position-wise feed-forward layer
    "att_dropout": 0.1,  # dropout for the multiheadattention module
    "dc_dropout": 0.1,  # dropout for the decoder block
    "hidden_act": "swish",  # activation function for the hidden layers (attention layers use ReLU)
    "epsilon": 1e-8,
    "weight_decay": 0.01,
    "beta1": 0.9,
    "beta2": 0.999,
}
model = TransformerEncoder(hparams)

# Sanity Check
Check if returns values and if the output looks OK.

In [3]:
batch = {
    'tokens': torch.randint(0, model.hparams.vocab_size, (model.hparams.batch_size, model.hparams.max_length)),
    'abspos': torch.arange(0, model.hparams.max_length).unsqueeze(0).repeat(model.hparams.batch_size, 1),
    'age': torch.randint(0, 100, (model.hparams.batch_size, 1)).repeat(1, model.hparams.max_length),
    'padding_mask': torch.zeros(model.hparams.batch_size, model.hparams.max_length),
    'targets': torch.randint(0, 2, (model.hparams.batch_size, 1)).float(),
}

In [4]:
batch

{'tokens': tensor([[98, 24, 87, 64, 50, 48, 19, 68,  1, 94,  8, 91, 54, 69,  4, 46, 93, 87,
          86,  1, 73, 65, 22, 44, 14, 60, 64, 25, 14, 34],
         [91, 39, 42, 26, 28, 61, 66, 44, 77, 62,  4,  3, 56,  2, 51, 11, 94, 60,
          28, 76, 97, 56, 38, 67, 68, 82, 60, 51, 21, 26],
         [ 9, 33, 31, 59, 12, 30, 69, 24, 84, 56, 50, 47, 75,  0, 58, 58, 54, 99,
          99, 65, 13,  4, 44, 66, 60, 56,  1, 49, 44, 92],
         [67, 63, 62, 46, 43,  5, 67, 58, 70, 17,  5, 30, 89, 72, 45, 91, 16, 26,
           0, 93, 72, 61,  7, 48, 21, 31,  1, 75, 42, 83]]),
 'abspos': tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

In [5]:
model(batch)

{'logits': tensor([[0.4836],
         [0.4518],
         [0.7592],
         [0.6978]], grad_fn=<AddmmBackward0>),
 'preds': tensor([[0.6186],
         [0.6111],
         [0.6812],
         [0.6677]], grad_fn=<SigmoidBackward0>)}

In [6]:
model.training_step(batch, 0)

/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/pytorch_lightning/core/module.py:447: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(0.6019, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [7]:
model.on_train_epoch_end()

### Full pipeline would be something like

In [8]:
from dataloaders.synthetic import SyntheticDataModule

dataloader = SyntheticDataModule(num_samples=1000, max_length=hparams['max_length'],
                                  batch_size=hparams['batch_size'], vocab_size=hparams['vocab_size'])

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

model_checkpoint = ModelCheckpoint(monitor='val/ap', save_top_k=2, save_last=True, mode='max')
early_stopping = EarlyStopping(monitor='val/ap', patience=5, mode='max')
logger = TensorBoardLogger("lightning_logs", name="cls_logs")

trainer = Trainer(max_epochs=30,
                accelerator="cpu",   ### change to "cuda" or "gpu" or 'msp'
                limit_train_batches=0.5,
                logger=logger,
                accumulate_grad_batches=4,
                num_sanity_val_steps=8,
                callbacks = [model_checkpoint, early_stopping],
                check_val_every_n_epoch=1)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [10]:
trainer.fit(model, dataloader)


   | Name        | Type                   | Params | Mode 
----------------------------------------------------------------
0  | transformer | Transformer            | 79.5 K | train
1  | decoder     | CLS_Decoder            | 2.4 K  | train
2  | loss        | BCEWithLogitsLoss      | 0      | train
3  | train_loss  | MeanMetric             | 0      | train
4  | val_loss    | MeanMetric             | 0      | train
5  | test_loss   | MeanMetric             | 0      | train
6  | train_acc   | BinaryAccuracy         | 0      | train
7  | val_acc     | BinaryAccuracy         | 0      | train
8  | test_acc    | BinaryAccuracy         | 0      | train
9  | train_mcc   | BinaryMatthewsCorrCoef | 0      | train
10 | val_mcc     | BinaryMatthewsCorrCoef | 0      | train
11 | test_mcc    | BinaryMatthewsCorrCoef | 0      | train
12 | train_ap    | BinaryAveragePrecision | 0      | train
13 | val_ap      | BinaryAveragePrecision | 0      | train
14 | test_ap     | BinaryAveragePrecision | 0    

/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0:   2%|▏         | 2/125 [00:00<00:01, 79.75it/s, v_num=0]

/Users/carlomarx/.local/share/virtualenvs/project2vec-D-jEdA35-python/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples found in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 0: 100%|██████████| 125/125 [00:03<00:00, 39.03it/s, v_num=0]

RuntimeError: Early stopping conditioned on metric `val_mcc` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train/loss`, `train/loss_step`, `train/acc`, `train/acc_step`, `train/mcc`, `train/mcc_step`, `train/ap`, `train/ap_step`, `val/loss`, `val/loss_epoch`, `val/acc`, `val/acc_epoch`, `val/mcc`, `val/mcc_epoch`, `val/ap`, `val/ap_epoch`, `train/loss_epoch`, `train/acc_epoch`, `train/mcc_epoch`, `train/ap_epoch`